In [ ]:
# 1. Import necessary libraries
import os
import numpy as np
import qcodes as qc
from qcodes.dataset import (
    Measurement,
    do1d,
    initialise_or_create_database_at,
    load_or_create_experiment,
)
from qcodes.station import Station
from qcodes.tests.instrument_mocks import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

from qcodes.dataset import (
    Measurement,
    initialise_or_create_database_at,
    load_or_create_experiment,
    plot_dataset,
)

import matplotlib.pyplot as plt

# 2. Setting up the configuration
qc.logger.start_all_logging()

# 3. Initializing instruments
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dac2=DummyInstrument('dac2', gates=['ch1', 'ch2','ch3','ch4'])
dmm = DummyInstrumentWithMeasurement(name='dmm', setter_instr=dac)
station = qc.Station(dmm, dac)

# 4. Setting up a database
db_file_path = os.path.join(os.getcwd(), 'plottr_for_live_plotting_tutorial.db')
initialise_or_create_database_at(db_file_path)
exp = load_or_create_experiment(
    experiment_name='plottr_for_live_plotting_with_subsecond_refresh_rate',
    sample_name="no sample"
)

# 5. Setting up measurements
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))
meas.write_period = 0.1

# 6. Running the measurements
with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        # get_v = dmm.v1.get()
        get_v = np.random.randint(2, size=10)
        datasaver.add_result((dac.ch1, set_v), (dmm.v1, get_v))

    dataset = datasaver.dataset

In [ ]:
# 5. Setting up measurements
meas = Measurement(exp=exp)
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dac2.ch1)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,dac.ch2,dac2.ch1), paramtype='array')
meas.register_parameter(dmm.v2, setpoints=(dac.ch1,dac.ch2,dac2.ch1))
meas.write_period = 0.1

# 6. Running the measurements
with meas.run() as datasaver:
    for set_v2 in np.linspace(0, 100, 100):
        for set_v in np.linspace(0, 25, 100):
            dac.ch1(set_v)
            dac.ch2(set_v2)
            get_v2 = dmm.v2.get()
            data_array = np.random.randint(5, size=20)
            datasaver.add_result((dac.ch1, set_v),(dac.ch2, set_v2),(dac2.ch1,set_v), (dmm.v1, data_array), (dmm.v2, get_v2))

    dataset = datasaver.dataset

In [ ]:
points_map={
    'initial_point': [0.0, 0.0],
    'anchor_point': [-0.0, 0.0],
    'load_point':    [0.0, 0.0],
    'readout_point': [0.0, 0.0], 
    'control_point': [0.0, 0.0],}

qc.config.add('points_map', points_map, value_type='object', description='Mapping of points', default=None)

In [ ]:
dataset_1d = load_by_run_spec(experiment_name='plottr_for_live_plotting_tutorial', captured_run_id=10)

In [73]:
## How to save the configuration
import json

dataset.add_metadata(tag='userconfig', metadata=json.dumps(qc.config['user']))
json.loads(dataset.get_metadata('userconfig'))['points_map']

{'initial_point': [0.0, 0.0],
 'anchor_point': [-0.0, 0.0],
 'load_point': [0.0, 0.0],
 'readout_point': [0.0, 0.0],
 'control_point': [0.0, 0.0]}